<a href="https://colab.research.google.com/github/sara-yasmin14/ASD-Detection-Using-AI-ML/blob/main/GWO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
from openpyxl import Workbook
import os
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
class GWO:
    def __init__(self, num_wolves, max_iter):
        self.num_wolves = num_wolves
        self.max_iter = max_iter

    def fitness(self, wolf, image):
        threshold = wolf[0]
        _, thresh_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
        mean_intensity = np.mean(thresh_image)
        std_deviation = np.std(thresh_image)
        # We aim to maximize mean intensity and minimize standard deviation
        fitness_value = mean_intensity - std_deviation
        return fitness_value

    def update_wolf_position(self, wolf, wolves, image, iter):
        a = 2 * (1 - (iter / self.max_iter))
        r1, r2 = np.random.uniform(0, 1, 2)
        X1, X2 = wolves[np.random.randint(0, self.num_wolves, 2)]
        D = np.abs(r1 * X1 - wolf)
        X_new = wolf + a * D
        X_new = np.clip(X_new, 0, 255)  # Ensure the new position is within valid range
        return X_new

    def optimize(self, image):
        # Initialize wolf pack
        wolves = np.random.uniform(0, 255, (self.num_wolves, 1))

        # GWO algorithm
        for iter in range(self.max_iter):
            # Update wolf positions
            for i in range(self.num_wolves):
                wolves[i] = self.update_wolf_position(wolves[i], wolves, image, iter)

        # Get the best wolf (optimal threshold)
        best_wolf = wolves[np.argmax([self.fitness(wolf, image) for wolf in wolves])]
        return best_wolf[0]

In [ ]:
def extract_features(image, threshold):
    _, thresh_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    mean_intensity = np.mean(thresh_image)
    std_deviation = np.std(thresh_image)
    return mean_intensity, std_deviation

In [ ]:
def main():
    # Define directory containing images in Google Drive
    images_dir = '/content/drive/My Drive/PATIENTS/'  # Update this path to your images directory

    # Create a Workbook object
    wb = Workbook()
    ws = wb.active

    # Define column names for features
    ws.append(['Image Name', 'Optimal Threshold', 'Mean Intensity', 'Standard Deviation'])

    # Initialize GWO optimizer
    gwo = GWO(num_wolves=5, max_iter=100)

    # Iterate over image files
    for filename in os.listdir(images_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read image from file
            image_path = os.path.join(images_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Optimize threshold using GWO
            optimal_threshold = gwo.optimize(image)

            # Extract features using the optimal threshold
            mean_intensity, std_deviation = extract_features(image, optimal_threshold)

            # Append features to Excel sheet
            ws.append([filename, optimal_threshold, mean_intensity, std_deviation])

    # Save Excel file to Google Drive
    excel_file = '/content/drive/My Drive/GWO_features.xlsx'  # Update this path as needed
    wb.save(excel_file)
    print("Features extracted and saved to", excel_file)

# Step 6: Run the main function
if __name__ == "__main__":
    main()

Features extracted and saved to /content/drive/My Drive/GWO_features.xlsx
